In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=150)
lngs = np.random.uniform(low=-180.000, high=180.000, size=150)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [6]:
cities

['mar del plata',
 'bambous virieux',
 'huaicheng',
 'hobart',
 'buala',
 'cabedelo',
 'bredasdorp',
 'tual',
 'lar',
 'araguari',
 'coquimbo',
 'mahebourg',
 'chuy',
 'avarua',
 'kapaa',
 'hammerfest',
 'saint-philippe',
 'puerto ayora',
 'tsihombe',
 'roebourne',
 'cherskiy',
 'saskylakh',
 'bilibino',
 'vaini',
 'punta arenas',
 'audincourt',
 'carnarvon',
 'dikson',
 'castro',
 'vao',
 'saint george',
 'ushuaia',
 'asyut',
 'rikitea',
 'port hardy',
 'anadyr',
 'port alberni',
 'nome',
 'mataura',
 'naze',
 'aklavik',
 'butaritari',
 'port lincoln',
 'yellowknife',
 'bluff',
 'kodiak',
 'ormond beach',
 'huarmey',
 'provideniya',
 'kysyl-syr',
 'bubaque',
 'hermanus',
 'inhambane',
 'jamestown',
 'port alfred',
 'borujan',
 'talnakh',
 'atuona',
 'linxia',
 'nouadhibou',
 'sitka',
 'kenai',
 'komsomolskiy',
 'khatanga',
 'yumen',
 'saint anthony',
 'east london',
 'portland',
 'omutinskoye',
 'qaanaaq',
 'adrar',
 'vysokogornyy',
 'coromandel',
 'cairns',
 'taolanaro',
 'aripuana',

In [7]:
# Import the requests library.
import requests

# Import the API key.
from config import api_key

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
# Create an endpoint URL for a city.
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=cf2ef14b68979549746bd13a432c79cf


In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              "Wind Speed": city_wind,
                              "Country": city_country,
                              "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mar del plata
Processing Record 2 of Set 1 | bambous virieux
Processing Record 3 of Set 1 | huaicheng
Processing Record 4 of Set 1 | hobart
Processing Record 5 of Set 1 | buala
Processing Record 6 of Set 1 | cabedelo
Processing Record 7 of Set 1 | bredasdorp
Processing Record 8 of Set 1 | tual
Processing Record 9 of Set 1 | lar
Processing Record 10 of Set 1 | araguari
Processing Record 11 of Set 1 | coquimbo
Processing Record 12 of Set 1 | mahebourg
Processing Record 13 of Set 1 | chuy
Processing Record 14 of Set 1 | avarua
Processing Record 15 of Set 1 | kapaa
Processing Record 16 of Set 1 | hammerfest
Processing Record 17 of Set 1 | saint-philippe
Processing Record 18 of Set 1 | puerto ayora
Processing Record 19 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 20 of Set 1 | roebourne
Processing Record 21 of Set 1 | cherskiy
Processing Record 22 of Set 1 | saskylakh
Processing R

In [10]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mar Del Plata,-38.0023,-57.5575,64.40,94,90,11.50,AR,2021-04-08 20:41:30
1,Bambous Virieux,-20.3428,57.7575,80.01,78,40,11.50,MU,2021-04-08 20:43:39
2,Huaicheng,23.9051,112.1931,61.43,74,79,2.46,CN,2021-04-08 20:43:40
3,Hobart,-42.8794,147.3294,51.80,58,40,13.80,AU,2021-04-08 20:39:05
4,Buala,-8.1450,159.5921,76.75,94,100,9.04,SB,2021-04-08 20:43:40
5,Cabedelo,-6.9811,-34.8339,82.40,78,20,14.97,BR,2021-04-08 20:39:35
6,Bredasdorp,-34.5322,20.0403,59.00,94,7,3.44,ZA,2021-04-08 20:43:40
7,Tual,-5.6667,132.7500,83.19,72,70,8.03,ID,2021-04-08 20:43:41
8,Lar,26.2039,83.9717,79.66,14,2,4.18,IN,2021-04-08 20:43:41
9,Araguari,-18.6472,-48.1872,73.47,56,100,10.74,BR,2021-04-08 20:43:41


In [11]:
city_data_df.to_csv("../weather_database/weatherpy_database.csv")